In [2]:
import os
import pandas as pd
import numpy as np
import regex as re
from collections import defaultdict
from typing import Tuple, List, NewType
from tqdm.notebook import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import seaborn as sns 
# %config InlineBackend.figure_format = 'retina'

import Bio
from Bio.PDB import *
import warnings
warnings.filterwarnings('ignore')


import logging



In [3]:
BioStructure = NewType('BioStructure', Bio.PDB.Structure.Structure)
BioVector = NewType('BioVector', Bio.PDB.vectors.Vector)  
BioResidue = NewType('BioResidue', Bio.PDB.Residue.Residue)  


In [4]:
#######################################################
# Load EpitopeDB
#######################################################
def load_epitopedb(path):
    """This function loads the Epitopes from different tables
    """
    # Load different sheets from excel as df and then concatenate all df's vertically 
    df_ABC = pd.read_excel(path, sheet_name='ABC')
    df_DRB1 = pd.read_excel(path, sheet_name='DRB1')
    df_DQB1 = pd.read_excel(path, sheet_name='DQB1')
    return  pd.concat([df_ABC, df_DRB1, df_DQB1])

#######################################################
# clean EpitopeDB
#######################################################
def clean_eptopedb(EpitopeDB):
    """ Drop some unnecessary columns, split Alleles and parse the polymorphic residues by parse_polymorphicresidues function
    """
    EpitopeDB = EpitopeDB.drop(['Frequency', 'StructEpitope', 'AntibodyReactivity'], axis=1)
    EpitopeDB['Luminex Alleles'] = EpitopeDB['Luminex Alleles'].apply(lambda x: set([ _.strip() for _ in sorted(x.split(sep=','))]))
    EpitopeDB['All Alleles'] = EpitopeDB['All Alleles'].apply(lambda x: set([ _.strip() for _ in sorted(x.split(sep=','))]))
    # Parse the polymorphic residues
    EpitopeDB.PolymorphicResidues = EpitopeDB.PolymorphicResidues.apply(lambda x:parse_polymorphicresidues(x))
    return EpitopeDB

#######################################################
# Load PDB file
#######################################################
def load_hla_structure(HLA_Molecule:str, path):
    parser = PDBParser()
    return parser.get_structure(HLA_Molecule, path)

#######################################################
# HLA to file name
#######################################################
def hla_to_filename(hla:str):
    """ """
    locus, specificity = hla.split('*')
    filename = '_'.join([locus, *specificity.split(':')]) + '_V1.pdb'
    return re.split('\d', locus)[0], filename

#######################################################
# Find HLA molecule path
#######################################################
def find_HLAMolecule_path(locus:str, filename:str) -> str:
    """This function makes use of the locus and filename resulted from 'hla_to_filename' function """
    
    path = os.path.expanduser(f'~/Downloads/HLAMolecule/{locus[0:2]}') # get until the first 2 character of locus if exist
    pdb_files = [file for file in os.listdir(path) if filename.split('_V1.pdb')[0] in file ]
    if len(pdb_files) != 0:
        return  True, os.path.join(path, f'{pdb_files[0]}')
    else:
        return  False, ''

#######################################################
# Residue from polymorphic residues
#######################################################
def parse_polymorphicresidues(string:str)-> Tuple[str,str]:
    """get string and return the string and num of the input string"""
    r = re.compile("([0-9]+)([a-zA-Z]+)")
    return r.findall(string)   # This line is inspired by re.findall(r'([0-9]+)([a-zA-Z]+)', string) 



#######################################################
# Residue in short
#######################################################
def res_short(residue:BioResidue) -> str:
    """ Gets the a residue object and returns a short residue sequence_number + amino acide name code """
    
    resname = residue.get_resname()  # Residue Name
    res_code = Aminoacid_conversion.get(resname)
    res_num = residue.get_id()[1]  # Residue Number 
    return str(res_num) + res_code


Aminoacid_conversion = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
                     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
                     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
                     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

#######################################################
# Find the average location of Residue
#######################################################
def get_residue_avg_coord(residue:Tuple[int,str], structure:BioStructure, chain:str) -> BioVector:
    """ This function finds the average coordinate of residue by averaging all the atoms coordinates"""
    
    BioChain = structure[0][chain]
    res_num, res_code = int(residue[0]), residue[1]
    _residue = BioChain[res_num]
#     print(_residue.get_full_id())
    res_pdb = Aminoacid_conversion.get(_residue.get_resname(), 'Corresponding code of the amino-acide could not be found')
    try:
        assert res_code == res_pdb
    except AssertionError as e:
         logger.warning(f'Expected residue {res_code}, but got {res_pdb}, sequence number: {res_num}, chain: {chain},  HLA: {structure.get_id()}')
    atoms_coord = [atom.get_vector() for atom in _residue.get_atoms()]
    return np.array(atoms_coord).sum()/len(atoms_coord)

#######################################################
# Find the average location of Epitope
#######################################################
def get_epitope_avg_coord(Epitope:List[Tuple[int,str]], structure:BioStructure, HLA_chain:str) -> BioVector:
    """ This function finds the average coordinate of Epitope by averaging all the residues average coordinates"""

    residues_coord = [get_residue_avg_coord(residue, structure, HLA_chain) for residue in Epitope]
    return np.array(residues_coord).sum()/len(residues_coord)

#######################################################
# Difference of two vectors
#######################################################
def distance_3d(vector1:BioVector, vector2:BioVector) -> int:
    x_diff = vector1[0] - vector2[0]
    y_diff = vector1[1] - vector2[1]
    z_diff = vector1[2] - vector2[2]
    return np.sqrt(x_diff**2 + y_diff**2 + z_diff**2)


#######################################################
# chain functions for calculating distances
#######################################################

def pre_selecttion(locus:str) -> Tuple[Tuple[int, str], str, str]:
    HLA_tail_residue = {'A': (196, 'D'), 'B': (196, 'D'), 'C': (196, 'D'), 'DRB': (167, 'S'), 'DQA': (167, 'R'), 'DQB': (167, 'R')} 
    # In HLA DQB1*, either (167, 'R') or (167, 'S') were noticed to be the tails
    HLA_tail_residue = HLA_tail_residue.get(locus, 'Locus not in HLA_tail')  # Tails: DR: 167S, 109L, DQ:167R
    HLA_tail_chain = {'A': 'A', 'B': 'A', 'C': 'A', 'DRB': 'B', 'DQA': 'B', 'DQB':'B'}
    HLA_chain = {'A': 'A', 'B': 'A', 'C': 'A', 'DRB': 'B', 'DQA': 'A', 'DQB':'B'}
    return HLA_tail_residue, HLA_tail_chain.get(locus, 'tail_chain not found!'), HLA_chain.get(locus, 'chain not found!')


def get_distance(poly_residues:List[str], structure:BioStructure, locus:str, hla:str) -> int:
    """ Locus:['A', 'B', 'C', 'DR', 'DQ'] should be max 2 letters
    """
    
    HLA_tail_residue, HLA_tail_chain, HLA_chain = pre_selecttion(locus)
    tail_coord = get_residue_avg_coord(HLA_tail_residue, structure, HLA_tail_chain)
    epitope_coord = get_epitope_avg_coord(poly_residues, structure, HLA_chain)
    return round(distance_3d(epitope_coord, tail_coord), 3)

# def get_distance_except(poly_residues:List[str], structure:BioStructure, locus:str, hla:str) -> int:
#     """ Locus:['A', 'B', 'C', 'DR', 'DQ'] should be max 2 letters
#     """
    
#     HLA_tail_residue, HLA_tail_chain, HLA_chain = (157,'T'), 'A', 
#     tail_coord = get_residue_avg_coord(HLA_tail_residue, structure, HLA_tail_chain)
#     epitope_coord = get_epitope_avg_coord(poly_residues, structure, HLA_chain)
#     return round(distance_3d(epitope_coord, tail_coord), 3)

def find_distances(EpitopeDB:pd.DataFrame) -> dict:
    
    hla_exceptions = ['DRB1*03:03', 'DRB1*09:02', 'A*02:06'] #'DQA1*05:01', 'DQA1*02:01','A*02:06']
    Epitope_distance = defaultdict(list)
    for i in tqdm(range(0, len(EpitopeDB))): #len(EpitopeDB)
        distance_info = defaultdict(int)
        Epitope = EpitopeDB.iloc[i].Epitope
        for hla in EpitopeDB.iloc[i]['Luminex Alleles']:
            if hla in hla_exceptions:
                logger.warning(f'Skipped hla: {hla}')
                continue
            locus, filename = hla_to_filename(hla)
            pdb_exist, pdb_path = find_HLAMolecule_path(locus, filename)
            if pdb_exist: 
                structure = load_hla_structure(hla, pdb_path)
                poly_residues = EpitopeDB.iloc[i].PolymorphicResidues
                try: 
                    distance_info[hla] =  get_distance(poly_residues, structure, locus, hla)
                except KeyError as e:
                    logger.error(f'Epitope {poly_residues} HLA {structure.get_id()} "KeyError" {e}')
        Epitope_distance[Epitope].append(distance_info)
    return Epitope_distance


def write_dsitance_df(EpitopeDB:pd.DataFrame) -> pd.DataFrame:
    Epitope_distance = find_distances(EpitopeDB)
    df = pd.DataFrame(Epitope_distance).T\
                                       .reset_index()\
                                       .rename(columns={'index':'Epitope', 0:'distance [A]'})
    df['mean_distance [A]'] =  df['distance [A]'].apply(lambda x: np.array([_[1] for _ in x.items()]).mean())
    df['std_distance [A]'] =  df['distance [A]'].apply(lambda x: np.array([_[1] for _ in x.items()]).std())
    return EpitopeDB.merge(df, on='Epitope')


In [5]:
# new = '~/UMCUtrecht/ProcessedData/20200504_EpitopevsHLA.xlsx' # Path to the new EpitopeDB scraped in May 2020.
new = 'EpitopevsHLA_clean.xlsx'
EpitopeDB = load_epitopedb(new)
EpitopeDB = clean_eptopedb(EpitopeDB)
# EpitopeDB = EpitopeDB.set_index('Epitope')

# Run the disance script

In [ ]:
# %%timeit

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
    
logger = logging.getLogger(__name__)
logging.basicConfig(filename= 'ep_distance.log',
                    filemode = 'w',
                    format= '%(name)s - %(levelname)s - %(message)s',
                    level=logging.DEBUG,
                   )


# dictionary = find_distances(EpitopeDB)
EpitopeDB_new = write_dsitance_df(EpitopeDB)
# EpitopeDB_new.to_pickle('EpitopevsHLA_distance.pickle')
# EpitopeDB_new.to_csv('EpitopevsHLA_distance.csv')

In [85]:
EpitopeDB.iloc[221,:].PolymorphicResidues = [(274,'W')]
EpitopeDB.iloc[219,:]
# EpitopeDB.iloc[219]

Epitope                                                            275EL
ElliPro Score                                                       High
PolymorphicResidues                                           [(274, W)]
Luminex Alleles        {A*36:01, A*11:02, A*30:01, A*01:01, A*03:01, ...
All Alleles            {A*01:23, A*11:03, A*30:23, A*11:26, A*11:19, ...
Name: 219, dtype: object

In [114]:
Epitope_distance = defaultdict(list)
for i in [219, 220, 221]:
    distance_info = defaultdict(int)
    Epitope = EpitopeDB.iloc[i].Epitope
    for hla in EpitopeDB.iloc[i]['Luminex Alleles']:
        locus, filename = hla_to_filename(hla)
        pdb_exist, pdb_path = find_HLAMolecule_path(locus, filename)
        if pdb_exist: 
            structure = load_hla_structure(hla, pdb_path)
            poly_residues = EpitopeDB.iloc[i].PolymorphicResidues
            try: 
                distance_info[hla] =  get_distance(poly_residues, structure, locus, hla)
            except KeyError as e:
                logger.error(f'Epitope {poly_residues} HLA {structure.get_id()} "KeyError" {e}')
    Epitope_distance[Epitope].append(distance_info)
Epitope_distance

defaultdict(list,
            {'275EL': [defaultdict(int,
                          {'A*36:01': 15.817,
                           'A*11:02': 15.834,
                           'A*30:01': 16.115,
                           'A*01:01': 15.855,
                           'A*03:01': 17.042,
                           'A*30:02': 16.611,
                           'A*11:01': 16.375})],
             '275G': [defaultdict(int,
                          {'C*08:02': 16.661,
                           'C*05:01': 16.733,
                           'C*08:01': 16.151})],
             '275K': [defaultdict(int,
                          {'C*04:03': 16.97,
                           'C*18:02': 16.274,
                           'C*04:01': 16.817,
                           'B*73:01': 16.117,
                           'C*17:01': 17.003})]})

In [116]:
np.array(list(Epitope_distance['275K'][0].values())).mean()

16.6362

In [6]:
EpitopeDB_new.to_pickle('./Database Versions/20200804_EpitopevsHLA_distance.pickle')
EpitopeDB_new.to_csv('./Database Versions/20200804_EpitopevsHLA_distance.csv')

# Epitope Distance Analysis

In [51]:
Ep_dist_df = pd.read_pickle('./Database Versions/20200804_EpitopevsHLA_distance.pickle')
Ep_dist_df.head(5)
Ep_dist_df.iloc[221]

Epitope                                                               6C
ElliPro Score                                                       High
PolymorphicResidues                                             [(6, C)]
Luminex Alleles                                             {DRB5*02:02}
All Alleles            {DRB5*02:03, DRB5*02:04, DRB1*13:112, DRB5*02:...
distance [A]                                      {'DRB5*02:02': 61.832}
mean_distance [A]                                                 61.832
std_distance [A]                                                       0
Name: 224, dtype: object

# Manual Distance Assignment

'3p': 62A

'275EL' ['274W']: 16.23

'275G' ['274W']: 16.51 while found to be 19

'95F' is discarded due to the absence of HLA C*07:04 molecule to locate the epitope on it!

In [45]:
Ep_dist_df.at[Ep_dist_df.Epitope=='3P', 'mean_distance [A]'] = 62
Ep_dist_df.at[Ep_dist_df.Epitope=='275EL', 'mean_distance [A]'] = 16.23
Ep_dist_df.at[Ep_dist_df.Epitope=='275G', 'mean_distance [A]'] = 16.51

In [52]:
Ep_dist_df[Ep_dist_df['mean_distance [A]'].isna()]

,Epitope,ElliPro Score,PolymorphicResidues,Luminex Alleles,All Alleles,distance [A],mean_distance [A],std_distance [A]


In [48]:
Ep_dist_df.dropna(inplace=True)

In [49]:
assert Ep_dist_df[Ep_dist_df['mean_distance [A]'].isna()].shape[0] == 0

In [50]:
Ep_dist_df.to_pickle('./Database Versions/20200804_EpitopevsHLA_distance.pickle')